### compare some ways to impute the month price matrix

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import mean_absolute_error

In [2]:
listings = pd.read_csv('seattle_airbnb_data.dir/listings.csv')
calendar = pd.read_csv('seattle_airbnb_data.dir/calendar.csv')

In [3]:
def filter_and_combine():
    condition_1 = (listings.room_type == 'Entire home/apt')
    condition_2 = (listings.property_type.isin(['House','Apartment','Townhouse','Condominium','Loft']))
    condition_3 = listings.first_review.apply(lambda x: int(x.split('-')[0])<=2015 if isinstance(x,str) else False)
    condition_4 = listings.last_review.apply(lambda x: int(x.split('-')[0])>=2015 if isinstance(x,str) else False)
    listings_filtered = listings.loc[ condition_1 & condition_2 & condition_3 & condition_4, 
                                     ['id','zipcode','bedrooms','host_since','first_review','last_review']].copy()
    
    # filtered the calendar so it has the same listing_id
    calendar_filtered = calendar[(calendar.available == 't') & 
                                 (calendar.listing_id.isin(listings_filtered.id))].copy()
    calendar_filtered['price'] = calendar_filtered['price'].str.lstrip('\$').replace('\,','',regex=True).astype(float)
    calendar_filtered['month'] = calendar_filtered.date.apply(lambda x: x.split('-')[1])
    month_price = calendar_filtered.groupby(['listing_id','month']).agg({'price':'mean'}).unstack(level=1)
    month_price.columns = month_price.columns.get_level_values(1)
    
    return listings_filtered, calendar_filtered, month_price

In [4]:
listings_filtered, calendar_filtered, month_price = filter_and_combine()

In [5]:
month_price

month,01,02,03,04,05,06,07,08,09,10,11,12
listing_id,,,,,,,,,,,,
5682,49.636364,52.826087,53.000000,53.000000,56.640000,60.000000,60.000000,60.000000,53.133333,48.580645,48.533333,48.645161
6606,92.666667,92.758621,92.580645,93.333333,92.580645,92.666667,93.225806,92.580645,93.000000,92.903226,92.666667,93.225806
9531,165.000000,NaN,NaN,165.000000,175.000000,185.000000,NaN,NaN,NaN,165.000000,165.000000,165.000000
9534,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,NaN,NaN,125.000000,125.000000,125.000000,125.000000
9596,121.071429,121.379310,121.290323,121.666667,121.290323,121.333333,121.612903,121.290323,121.500000,121.451613,121.333333,121.612903
...,...,...,...,...,...,...,...,...,...,...,...,...
9866984,101.000000,97.105263,97.352941,103.888889,105.000000,103.571429,110.476190,102.741935,104.000000,103.709677,103.000000,104.677419
9908591,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN
9924107,114.000000,114.137931,113.870968,115.000000,113.870968,114.000000,114.444444,113.870968,114.500000,114.354839,114.000000,114.838710


In [6]:
X = month_price.values

In [7]:
np.isnan(X).all(axis=1).sum()

0

In [8]:
def test(X, random_state=123, fake_rate=0.25):
    
    np.random.seed(random_state)
    
    mask_na = np.isnan(X)
    mask_fake_na = np.where((np.random.rand(*X.shape)<fake_rate) & (mask_na==False), True, False)
    X_fake = X.copy()
    X_fake[mask_fake_na] = np.nan
    
    # one more check to make sure at least 1 element is not nan in each row
    mask_fake_na[np.isnan(X_fake).all(axis=1)] = False
    X_fake = X.copy()
    X_fake[mask_fake_na] = np.nan

    # now do some test
#     X_imputed = SimpleImputer(strategy='mean').fit_transform(X_fake)
#     print('mean on column', mean_absolute_error(X_imputed[mask_fake_na], X[mask_fake_na]))
    X_imputed = SimpleImputer(strategy='mean').fit_transform(X_fake.T)
    X_imputed = X_imputed.T
    print('mean on row', mean_absolute_error(X_imputed[mask_fake_na], X[mask_fake_na]))
    
#     X_imputed = SimpleImputer(strategy='median').fit_transform(X_fake)
#     print('median on column', mean_absolute_error(X_imputed[mask_fake_na], X[mask_fake_na]))
    X_imputed = SimpleImputer(strategy='median').fit_transform(X_fake.T)
    X_imputed = X_imputed.T
    print('median on row', mean_absolute_error(X_imputed[mask_fake_na], X[mask_fake_na]))
    
    print('knn stuff')
    for n_neighbors in [3,5,10,20,50]:
        X_imputed = KNNImputer(n_neighbors=n_neighbors).fit_transform(X_fake)
        print(n_neighbors, mean_absolute_error(X_imputed[mask_fake_na], X[mask_fake_na]))

In [9]:
for seed in [1,2,3,4,5]:
    X_fake = test(X, random_state=seed)

mean on row 15.29820687759649
median on row 14.02045585184062
knn stuff
3 12.503159044193788
5 12.163824747154901
10 12.085662211379505
20 12.812946959456514
50 15.510522471260412
mean on row 15.762010450619789
median on row 14.922357018205542
knn stuff
3 12.535222850375538
5 12.213323506937144
10 12.126741574665171
20 12.526935638732317
50 14.762166012613635
mean on row 16.57669306995177
median on row 15.747418561110376
knn stuff
3 13.047835737266178
5 12.878762770001954
10 12.649340428926095
20 13.043403496638712
50 15.48958557370268
mean on row 16.07182981655488
median on row 15.059466618145509
knn stuff
3 12.322678011851895
5 11.983051355772888
10 11.906483664833793
20 12.548440031507527
50 14.690240692753532
mean on row 16.26649590107067
median on row 15.518317014094219
knn stuff
3 13.044648062020343
5 12.908191820176539
10 12.614764754282874
20 13.101332548006551
50 15.289012997814888


### so knn with n_neighbors = 10 seems doing the best; it can get to a median absolute error of 12-13, which seems fine